<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/hotel_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# libs

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gspread_dataframe import get_as_dataframe

In [2]:
#goog libs

# !pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('recommender_india').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
ihotel = get_as_dataframe(worksheet=worksheet, header=0)

In [78]:
ihotel.fillna('0')
ihotel.head(6)

,address,city,country,crawl_date,hotel_brand,hotel_description,hotel_facilities,hotel_star_rating,image_count,latitude,locality,longitude,pageurl,property_id,property_name,property_type,province,qts,room_count,room_type,similar_hotel,site_review_count,site_review_rating,site_stay_review_rating,sitename,special_tag,state,uniq_id,zone,clean_desc
0,"KHIRSU, 246147 Pauri, India â€“ Great location -",pauri,India,9/1/2016,NaN,Khirsu By GMVN offers accommodation in Pauri. ...,Bathroom:Toilet paper|Linen|Towels|Bathroom|To...,NaN,3,30.123749,NaN,78.7938,http://www.booking.com/hotel/in/khirsu-by-gmvn...,1742658,Khirsu By GMVN,204,NaN,2016-09-01 11:52:38 +0000,18,Economy Double Room,Hotel Mandakini,NaN,NaN,NaN,http://www.booking.com/,Share,Uttarakhand,a5ea72415f8007fcbe65759830fdd3d9,NaN,Khirsu By GMVN offers accommodation in Pauri. ...
1,"Kaathadimattam, Balacola Post, NEAR Siva Tea F...",ooty,India,9/1/2016,NaN,"Situated in Ooty in the Tamil Nadu Region, 8 k...",Bathroom:Toilet paper|Linen|Towels|Bidet|Towel...,3 stars,NaN,11.329595,NaN,76.6867,http://www.booking.com/hotel/in/global-village...,1755986,Global Village Ooty,212,NaN,2016-09-01 11:52:38 +0000,10,British Empire Chalet,Treebo Yantra Leisures|Western Valley Resorts|...,5,7.6,Location:8.5|Staff:8|Cleanliness:7.5|Comfort:7...,http://www.booking.com/,Share,Tamil Nadu,7e0b055417271bbd9dae902f3e231ed4,NaN,"Situated in Ooty in the Tamil Nadu Region, 8 k..."
2,"PIPALKOTI, 246472 PÄ«palkoti, India â€“ Show map",pÄ«palkoti,India,9/1/2016,NaN,TRH Pipalkoti offers accommodation in PÄ«palko...,Bathroom:Toilet paper|Linen|Towels|Bathroomâ€¢...,NaN,4,30.429540,NaN,79.4281,http://www.booking.com/hotel/in/trh-pipalkoti....,1741747,TRH Pipalkoti,204,NaN,2016-09-01 11:52:38 +0000,28,Economy Double Room,TRH Joshimath (New),NaN,NaN,NaN,http://www.booking.com/,Share,Uttarakhand,72c0af09827bbb620365aa5df523ba1d,NaN,TRH Pipalkoti offers accommodation in Ppalkoti...
3,"1 KARIYIL HOUSE KUMARAKOM NORTH PO KOTTAYAM, 6...",kumarakom,India,9/1/2016,NaN,"Swasti house boat 2 is located in Kumarakom, 3...",Bathroom:Toilet paper|Towels|Bath|Showerâ€¢Bed...,NaN,2,9.616057,NaN,76.4283,http://www.booking.com/hotel/in/swasti-house-b...,1747392,Swasti house boat 2,204,NaN,2016-09-01 11:52:38 +0000,1,Deluxe Room,NaN,NaN,NaN,NaN,http://www.booking.com/,Share,Kerala,eb6fd33d99aa4a8088caa8f3ecb08275,NaN,"Swasti house boat 2 is located in Kumarakom, 3..."
4,"Kavanattinkara, 686563 Kumarakom, India â€“ Sh...",kumarakom,India,9/1/2016,NaN,"Amrutham Houseboat 2 is set in Kumarakom, 5 km...",Bathroom:Toilet paper|Linen|Towels|Towels/Shee...,NaN,NaN,9.632854,NaN,76.4236,http://www.booking.com/hotel/in/amrutham-house...,1744962,Amrutham Houseboat 2,215,NaN,2016-09-01 11:52:38 +0000,2,Mobile Home,Mandala Beach House & Cottages,NaN,NaN,NaN,http://www.booking.com/,Share,Kerala,0814d9af7ad808863c04db8f30437c57,NaN,"Amrutham Houseboat 2 is set in Kumarakom, 5 km..."
5,"15 A Arani House Ooty, 643002 Ooty, India â€“ ...",ooty,India,9/1/2016,NaN,Located 3.4 km from Ooty Lake and 800 m from H...,Miscellaneous:Non-smoking roomsâ€¢Bathroom:Toi...,NaN,12,11.413460,NaN,76.7186,http://www.booking.com/hotel/in/the-hill-side....,1756349,The Hill Side,201,NaN,2016-09-01 11:52:38 +0000,20,Standard Room in Apartment,Two Seas Residence,81,8.4,NaN,http://www.booking.com/,Share,Tamil Nadu,cd2caff07410c06831e50a5ed0807f6c,NaN,Located 3.4 km from Ooty Lake and 800 m from H...


In [65]:
ihotel['clean_desc'] = ihotel.hotel_description
ihotel.clean_desc

0       Khirsu By GMVN offers accommodation in Pauri. ...
1       Situated in Ooty in the Tamil Nadu Region, 8 k...
2       TRH Pipalkoti offers accommodation in PÄ«palko...
3       Swasti house boat 2 is located in Kumarakom, 3...
4       Amrutham Houseboat 2 is set in Kumarakom, 5 km...
                              ...                        
5997    Featuring free WiFi and a restaurant, Hotel Or...
5998    Located 4.1 km from The Great India Place in N...
5999    One of our bestsellers in Mumbai! Hotel Silver...
6000    Offering a barbecue, children's playground and...
6001    One of our bestsellers in Candolim! This prope...
Name: clean_desc, Length: 6002, dtype: object

In [66]:
ihotel['clean_desc'] = ihotel.hotel_description

ihotel.clean_desc = ihotel.clean_desc.str.replace(r'[^a-zA-Z0-9.,\' ]?',r'', regex=True)
ihotel.clean_desc

0       Khirsu By GMVN offers accommodation in Pauri. ...
1       Situated in Ooty in the Tamil Nadu Region, 8 k...
2       TRH Pipalkoti offers accommodation in Ppalkoti...
3       Swasti house boat 2 is located in Kumarakom, 3...
4       Amrutham Houseboat 2 is set in Kumarakom, 5 km...
                              ...                        
5997    Featuring free WiFi and a restaurant, Hotel Or...
5998    Located 4.1 km from The Great India Place in N...
5999    One of our bestsellers in Mumbai Hotel Silver ...
6000    Offering a barbecue, children's playground and...
6001    One of our bestsellers in Candolim This proper...
Name: clean_desc, Length: 6002, dtype: object

In [67]:
tfidf = TfidfVectorizer(stop_words='english')
ihotel.clean_desc = ihotel.clean_desc.fillna('')

tfidf_matrix = tfidf.fit_transform(ihotel.clean_desc)


In [68]:
tfidf_matrix.shape

(6002, 5425)

In [69]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape
# (cosine_sim<0).sum()

(6002, 6002)

In [79]:
indices = pd.Series(ihotel.index, index=ihotel.property_name).drop_duplicates()

In [80]:
indices

property_name
Khirsu By GMVN                     0
Global Village Ooty                1
TRH Pipalkoti                      2
Swasti house boat 2                3
Amrutham Houseboat 2               4
                                ... 
Hotel Orient Inn                5997
Hotel Ascent Biz                5998
Hotel Silver Moon               5999
Ranches Jungle Beats Resorts    6000
Zense Resort                    6001
Length: 6002, dtype: int64

In [81]:
indices.value_counts().max()

1

In [82]:
idx = indices['TRH Pipalkoti']
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:11]
movie_indices =  [i[0] for i in sim_scores]
movie_indices

[4433, 4662, 4672, 418, 331, 437, 4694, 163, 232, 336]

In [85]:
idx

2

In [87]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return ihotel.clean_desc.iloc[movie_indices] 

In [88]:
get_recommendations('TRH Pipalkoti')

4433    Vista Rooms near B.B.C Tower offers accommodat...
4662    Vista Rooms near B.B.C Tower offers accommodat...
4672    Vista Rooms near B.B.C Tower offers accommodat...
418     Featuring free WiFi throughout the property, B...
331     Featuring free WiFi, Mickey Homestay offers pe...
437     Featuring free WiFi, Mickey Homestay offers pe...
4694    Featuring free WiFi throughout the property, V...
163     Featuring free WiFi, a restaurant and a terrac...
232     Suchee's Cliff House offers petfriendly accomm...
336     Suchee's Cliff House offers petfriendly accomm...
Name: clean_desc, dtype: object